## 미래에셋 빅데이터 콘테스트
### 보험금 청구 분류
### 분류기 기법 적용

In [1]:
#라이브러리 불러오기
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler

train = pd.read_csv("D:/thon/data/mirae/classification/train.csv")
test = pd.read_csv("D:/thon/data/mirae/classification/test.csv")
train_dummy = pd.read_csv("D:/thon/data/mirae/classification/train_dummy.csv")
test_dummy = pd.read_csv("D:/thon/data/mirae/classification/test_dummy.csv")

#필요없는 항목 삭제
test_dummy.drop(['Unnamed: 0'], axis = 1, inplace = True)
test_dummy.drop(['ID'], axis = 1, inplace = True)
test_dummy.drop(['base_ym'], axis = 1, inplace = True)

train_dummy.drop(['Unnamed: 0'], axis = 1, inplace = True)
train_dummy.drop(['ID'], axis = 1, inplace = True)
train_dummy.drop(['base_ym'], axis = 1, inplace = True)

### 먼저 10% sample로 분류기 성능 평가

In [2]:
#10% 시험 수행 준비
t1 = train_dummy.sample(frac = 0.1)

#X와 Y로 변수 구별
x_t1 = t1.drop(['target'], axis = 1)
y_t1 = t1['target']

#훈련, 테스트 데이터로 나누기 (default로 train 75%, test 25%)
x_train, x_test, y_train, y_test = train_test_split(x_t1, y_t1, random_state = 0)

### 모델 성능 평가
###  
#### RandomForest, Bagging, AdaBoost, GradientBoost, KNN, MLP

In [3]:
#랜덤포레스트
classifier_randomforest = RandomForestClassifier()
classifier_randomforest.fit(x_train, y_train)
classifier_randomforest.score(x_test, y_test)

0.8281299608424172

In [4]:
#배깅
classifier_bagging = BaggingClassifier()
classifier_bagging.fit(x_train, y_train)
classifier_bagging.score(x_test, y_test)

0.7928881363107207

In [5]:
#AdaBoost
classifier_adaboost = AdaBoostClassifier()
classifier_adaboost.fit(x_train, y_train)
classifier_adaboost.score(x_test, y_test)

0.689914276643031

In [6]:
#gradientboost
classifier_gradientboost = GradientBoostingClassifier()
classifier_gradientboost.fit(x_train, y_train)
classifier_gradientboost.score(x_test, y_test)

0.7579638056937242

In [7]:
#knn
classifier_knn = KNeighborsClassifier()
classifier_knn.fit(x_train, y_train)
classifier_knn.score(x_test, y_test)

0.7577521430839242

In [8]:
#MLP
classifier_mlp = MLPClassifier()
classifier_mlp.fit(x_train, y_train)
classifier_mlp.score(x_test, y_test)

C:\Users\thon\anaconda3\envs\thon\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.7769076092708224

In [26]:
#랜덤포레스트
randomforest = RandomForestClassifier(random_state = 0, n_jobs = -1, class_weight = "balanced")
randomforest.fit(x_train, y_train)
randomforest.score(x_test, y_test)
pred = randomforest.predict(x_test)

In [ ]:
from imblearn.over_sampling import SMOTEsmote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_sample(X_train,y_train)print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

In [30]:
metrics.plot_confusion_matrix(y_test, pred, normalize=True)

AttributeError: 'function' object has no attribute 'plot_confusion_matrix'

In [31]:
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [35]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, pred))

[[1822  532   15]
 [ 510 5060  107]
 [  10  475  918]]


In [13]:
def modeling(model,x_train,x_test,y_train,y_test):
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    metrics(y_test,pred)

In [34]:
print('f1', metrics.f1_score(y_test,pred) )

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [32]:
print('accuracy', metrics.accuracy_score(y,pred) )

print('precision', metrics.precision_score(y,p) )

print('recall', metrics.recall_score(y,p) )

print('f1', metrics.f1_score(y,p) )

NameError: name 'y' is not defined

In [24]:
def metrics(y_test,pred):
    accuracy = metrics.accuracy_score(y_test,pred)
    precision = metrics.precision_score(y_test,pred)
    recall = metrics.recall_score(y_test,pred)
    f1 = metrics.f1_score(y_test,pred)
    roc_score = roc_auc_score(y_test,pred,average='macro')
    print('정확도 : {0:.2f}, 정밀도 : {1:.2f}, 재현율 : {2:.2f}'.format(accuracy,precision,recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1,roc_score,recall))

In [25]:
modeling(randomforest,x_train,x_test,y_train,y_test)

AttributeError: 'function' object has no attribute 'accuracy_score'

## 10% sample로 모델링한 결과 : RandomForest 가 가장 우수
### 이제 train_dummy 전체 데이터에 RandomForest 적용하기

In [5]:
#X와 Y로 변수 구별
x_train_dummy = train_dummy.drop(['target'], axis = 1)
y_train_dummy = train_dummy['target']

#타깃 변수 문자형 변환
y_train_dummy = y_train_dummy.astype(str)

#훈련, 테스트 데이터로 나누기
x_train, x_test, y_train, y_test = train_test_split(x_train_dummy, y_train_dummy, random_state = 0)

#randomforest 적용해서 분류기 성능 확인
rf = RandomForestClassifier()
model = rf.fit(x_train, y_train)
rf.score(x_test, y_test)

#### 랜덤포레스트 분류기 테스트 데이터에 적용

In [9]:
#test_dummy 데이터에 모델 적용
target_pred = model.predict(test_dummy)

#예측 결과 데이터프레임화
target = pd.DataFrame(target_pred, columns = ['target'])

#결과 csv 파일 저장
target.to_csv('D:/thon/data/mirae/classification/target_predict1.csv')